In [ ]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')
sys.path.append('..')

# Maths
import numpy as np

# Plotting
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.image as mplimg

# ML
from soap import compute_soap_density, reshape_soaps
from soap import rrw_neighbors, make_tuples

# Utilities
import h5py
import json
import project_utils as utils

# Import COSMO style toolkit
import cosmoplot.colorbars as cosmocbars
import cosmoplot.utils as cosmoutils
import cosmoplot.style as cosmostyle

cosmostyle.set_style('article')
colorList = cosmostyle.color_cycle

# Functions

In [ ]:
# 0: O-O
# 1: Si-O
# 2: Si-Si
species_pair_idx = 0
soap_idx = 0

def convert_species_idx(species_pair_idx):
    if species_pair_idx == 0:
        species_pair_label = 'OO'
    elif species_pair_idx == 1:
        species_pair_label = 'OSi'
    elif species_pair_idx == 2:
        species_pair_label = 'SiSi'
    else:
        species_pair_label = 'XX'
    return species_pair_label
 
def convert_soap_idx(soap_idx):
    if soap_idx == 0:
        soap_label = 'DEEM'
    elif soap_idx == 1:
        soap_label = 'IZA'
    else:
        soap_label = 'XXX'
    return soap_label

In [ ]:
#def make_plot(output, density, r_grid, p_grid, species_pair_label, neighbors=None):
# TODO: scatter points only for selected environments?
def make_plot(output, r_grid, p_grid, density, density_highlight=None,
              rrw_neighbors=None, idxs_neighbors=None, 
              species_pair_idx=None, species_pair_name=None,
              scatter=False, annotate=[]):
    
    annotations = []
    
    rx_grid, ry_grid, pz_grid = np.meshgrid(r_grid, r_grid, p_grid, indexing='ij')
    
    avg = np.mean(density)
    std = np.std(density)
    
    avg_highlight = np.mean(density_highlight)
    std_highlight = np.std(density_highlight)

    fig = go.Figure()
    fig.add_trace(
        go.Isosurface(
            x=rx_grid.flatten(),
            y=ry_grid.flatten(),
            z=pz_grid.flatten(),
            value=density.flatten(),
            coloraxis='coloraxis1',
            isomin=avg - 2.0*std,
            isomax=avg + 2.0*std,
            opacity=0.3,
            surface_count=4,
            caps=dict(x_show=False, y_show=False, z_show=False)
        )
    )
        
    if density_highlight is not None:
        fig.add_trace(
            go.Volume(
                x=rx_grid.flatten(),
                y=ry_grid.flatten(),
                z=pz_grid.flatten(),
                value=density_highlight.flatten(),
                coloraxis='coloraxis2',
                isomin=avg_highlight - 2.0*std_highlight,
                isomax=avg_highlight + 2.0*std_highlight,
                opacity=0.4,
                surface_count=2,
                caps=dict(x_show=False, y_show=False, z_show=False)
            )
        )

    if rrw_neighbors is not None and species_pair_idx is not None:
        atom_stencils = [[], [], []]
        for xyz in range(0, len(atom_stencils)):
            atom_stencils[xyz] = np.concatenate(
                [center[species_pair_idx][xyz].flatten() for center in rrw_neighbors]
            )

        if idxs_neighbors is not None:
            idxs = [[], [], []]
            for xyz in range(0, len(idxs)):
                idxs[xyz] = np.concatenate(
                    [center[species_pair_idx][xyz].flatten() for center in idxs_neighbors]
                )
            hovertemplate = 'x: %{x}<br>y: %{y}<br>z: %{z}<br>(i, j, k): %{text}'
            hovertext = ['{}'.format(i) for i in zip(idxs[0], idxs[1], idxs[2])]
        else:
            hovertemplate = 'x: %{x}<br>y: %{y}<br>z: %{z}'
            hovertext=None

        if scatter:
            fig.add_trace(
                go.Scatter3d(
                    x=atom_stencils[0],
                    y=atom_stencils[1],
                    z=atom_stencils[2],
                    name=f'{species_pair_name}',
                    mode='markers',
                    marker=dict(size=5, color='purple'),
                    hovertemplate=hovertemplate,
                    text=hovertext,
                    showlegend=True
                )
            )
        
        for center in annotate:
            annotations.append(dict(
                showarrow=True,
                x=atom_stencils[0][center],
                y=atom_stencils[1][center],
                z=atom_stencils[2][center],
                hovertext=f'(i, j, k): {hovertext}',
                text=f'{center}',
                arrowcolor='Black',
                arrowsize=1,
                arrowwidth=2,
                ax=50,
                ay=-50,
                xshift=5,
                yshift=5,
            ))
                
    # TODO: set scaling
    fig.update_layout(
        template='plotly_white',
        scene=dict(
            xaxis_title='r',
            yaxis_title='r\'',
            zaxis_title='w',
            annotations=annotations,
            camera=dict(
                eye=dict(x=-1.25, y=-1.25, z=1.25),
                projection=dict(type='orthographic')
            )
        ),
        legend=dict(
            x=0.0, y=1.0,
            xanchor='left', yanchor='top',
            itemsizing='constant'
        ),
        coloraxis1=dict(
            colorscale='RdBu',
            colorbar=dict(
                title=dict(text='Weights*Density', side='right'),
                x=-0.02, xanchor='right'
            ),
        ),
        coloraxis2=dict(
            colorscale='Purples',
            colorbar=dict(
                title=dict(text='Density', side='right')
            ),
        ),
        font=dict(size=16, family='Serif'),
        autosize=False, width=800, height=800
    )
    
    fig.show()

    #fig.write_html(f'{output}.html')
    #fig.write_image(f'{output}.png')

# Setup

In [ ]:
# Load SOAP cutoffs
with open('../../Processed_Data/soap_hyperparameters.json', 'r') as f:
    soap_hyperparameters = json.load(f)
    
cutoffs = soap_hyperparameters.pop('interaction_cutoff')

In [ ]:
n_Si_deem = np.loadtxt('../../Processed_Data/DEEM_10k/n_Si.dat', dtype=int)
n_Si_iza = np.loadtxt('../../Processed_Data/IZA_226/n_Si.dat', dtype=int)

In [ ]:
# Load train and test indices
idxs_deem_train = np.loadtxt('../../Processed_Data/DEEM_10k/train.idxs', dtype=int)
idxs_deem_test = np.loadtxt('../../Processed_Data/DEEM_10k/test.idxs', dtype=int)

n_deem_train = idxs_deem_train.size
n_deem_test = idxs_deem_test.size
n_deem = n_deem_train + n_deem_test

In [ ]:
idxs_iza_train = np.loadtxt('../../Processed_Data/IZA_226/train.idxs', dtype=int)
idxs_iza_test = np.loadtxt('../../Processed_Data/IZA_226/test.idxs', dtype=int)

In [ ]:
# Set grids
# TODO: load grids?
n_r_grid = 50
chunk_size_r = 10
r_grid = {}
for cutoff in cutoffs:
    r_grid[cutoff] = np.linspace(0.0, cutoff, n_r_grid)

n_p_grid = 50
chunk_size_p = 10
p_grid = np.linspace(-1.0, 1.0, n_p_grid)

In [ ]:
# Linear model setup
n_species = 2
#group_names = {'power': ['OO', 'OSi', 'SiSi', 
#                         'OO+OSi', 'OO+SiSi', 'OSi+SiSi',
#                         'OO+OSi+SiSi'], 
#               'radial': ['O', 'Si', 'O+Si']}
group_names = {'power': ['OO', 'OSi', 'SiSi'],
               'radial': ['O', 'Si']}

In [ ]:
model_dir = '../../Processed_Data/Models'

# TODO: load just from the 330k
deem_name = 'DEEM_10k'
iza_name = 'IZA_226'
deem_dir = f'../../Processed_Data/{deem_name}/Data'
iza_dir = f'../../Processed_Data/{iza_name}/Data'

# TMP: SODALITE

In [ ]:
from ase.io import read
from soap import librascal_soap

In [ ]:
cantons_iza = np.loadtxt('../../Raw_Data/GULP/IZA_226/cantons.txt', usecols=1, dtype=int)
RWY = np.nonzero(cantons_iza == 4)[0][0]

In [ ]:
# Load structure
sod = read('../../Raw_Data/SOD/sodalite.xyz', index=':')

In [ ]:
soaps_sod = {}
density_sod = {}
sod_neighbor_rrw = {}
sod_neighbor_idxs = {}
for cutoff in cutoffs:
    
    # Compute SOAPs
    soaps_sod[cutoff] = librascal_soap(
        sod, [14],
        **soap_hyperparameters,
        interaction_cutoff=cutoff,
        normalize=False,
        average=True
    )

    soaps_sod[cutoff] = np.mean(soaps_sod[cutoff], axis=0)
    
    linear_dir = f'{model_dir}/{cutoff}/Linear_Models/LSVC-LPCovR'

    # Load SOAPs
    deem_file = f'{deem_dir}/{cutoff}/soaps_power_full_avg_nonorm.hdf5'
    iza_file = f'{iza_dir}/{cutoff}/soaps_power_full_avg_nonorm.hdf5'    

    # Build average IZA and DEEM SOAP over the whole dataset instead of per-structure
    soaps_deem = utils.load_hdf5(deem_file, datasets=None, concatenate=True)

    soaps_iza = utils.load_hdf5(iza_file, datasets=None, concatenate=True)
    soaps_iza = np.delete(soaps_iza, RWY, axis=0)

    # Scale SOAPs to be compatible with the weights
    soaps_scale = np.std(np.concatenate((soaps_iza[idxs_iza_train], soaps_deem[idxs_deem_train])))
    
    soaps_sod[cutoff] /= soaps_scale

    soaps_sod[cutoff] = reshape_soaps(
        soaps_sod[cutoff], 3, 
        soap_hyperparameters['max_radial'], 
        soap_hyperparameters['max_angular']
    )

    # Compute density
    density_sod[cutoff] = compute_soap_density(
        soap_hyperparameters['max_radial'],
        soap_hyperparameters['max_angular'],
        cutoff, soaps_sod[cutoff], 
        r_grid[cutoff], p_grid, 
        chunk_size_r=10, chunk_size_p=10
    )
    
    sod_neighbor_rrw[cutoff], sod_neighbor_idxs[cutoff] = \
        rrw_neighbors(sod[0], [14], [8, 14], cutoff, self_interaction=True)

# Plot

In [ ]:
for cutoff in cutoffs:
    linear_dir = f'{model_dir}/{cutoff}/Linear_Models/LSVC-LPCovR'
    
    # TODO: load iza rrw indices
    # TODO: load deem rrw indices
    
    #for spectrum_type in ('power', 'radial'):
    for spectrum_type in ['power']:
        spectrum_name = spectrum_type.capitalize()        
        
        for gdx, group_name in enumerate(group_names[spectrum_type]):
            deem_file = f'{deem_dir}/{cutoff}/real_space_soaps_{spectrum_type}_full_avg_nonorm_{group_name}.hdf5'
            iza_file = f'{iza_dir}/{cutoff}/real_space_soaps_{spectrum_type}_full_avg_nonorm_{group_name}.hdf5'
            
            hdf5_deem = h5py.File(deem_file, 'r')
            hdf5_iza = h5py.File(iza_file, 'r')
            
            soaps_deem = hdf5_deem['0'][:]
            soaps_deem *= np.sum(n_Si_deem)
            soaps_iza = hdf5_iza['0'][:] * np.sum(n_Si_iza)
            soaps_avg = (soaps_iza + soaps_deem) / (np.sum(n_Si_iza) + np.sum(n_Si_deem))
            
            hdf5_deem.close()
            hdf5_iza.close()
            
            partial_density_sod = density_sod[cutoff][:, gdx, ...]
            
            # TODO: how to handle multiple decision functions?
            #for n_cantons in (2, 4):
            for n_cantons in [2]:
                
                output = f'../../Results/{cutoff}/real_space_overlap_{n_cantons}-class_{spectrum_type}_{group_name}'
                
                weights_dir = f'{linear_dir}/{n_cantons}-Class/{spectrum_name}/{group_name}'
                weights_file = f'{weights_dir}/real_space_svc_weights.hdf5'
                
                hdf5_weights = h5py.File(weights_file, 'r')
                weights = hdf5_weights['0'][:]
                
                hdf5_weights.close()
                
                overlap = weights * soaps_avg

                make_plot(
                    output, 
                    #weights.squeeze() * partial_density_sod.squeeze(), 
                    r_grid[cutoff], p_grid,
                    weights.squeeze(),
                    partial_density_sod.squeeze(), 
                    rrw_neighbors=sod_neighbor_rrw[cutoff],
                    idxs_neighbors=sod_neighbor_idxs[cutoff],
                    species_pair_idx=gdx,
                    species_pair_name=group_name,
                    scatter=True, annotate=[0, 4, 8]
                )
                
                #img = mplimg.imread(f'{output}.png')
                #plt.imshow(img)